In [66]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import sklearn.model_selection as sk

In [67]:
spring_csv = "/home/liujinzhao/anaconda3/output.csv"

In [68]:
datasets = np.loadtxt(fname=spring_csv,delimiter=',')

In [69]:
datasets = tf.convert_to_tensor(datasets)
N = datasets.shape[0].value
K = 10
Max_iter = 1000

In [70]:
cluster_assignments = tf.Variable(tf.zeros([N],dtype=tf.int64))
#have to change here
centroids = tf.random_crop(datasets,[10,50])#
#changed to under,you have to understand the meaning of Variable.
centroids = tf.Variable(tf.slice(datasets,[0,0],[K,50]))
centroids = tf.Variable(tf.slice(tf.random_shuffle(datasets),[0,0],[K,50]))

In [71]:
rep_centroids = tf.reshape(tf.tile(centroids,[N,1]),[N,K,50])
rep_points = tf.reshape(tf.tile(datasets,[1,K]),[N,K,50])
sum_squares = tf.reduce_sum(tf.square(rep_points - rep_centroids),reduction_indices=2)

In [72]:
best_centroids = tf.argmin(sum_squares,1)
did_assignment_changed = tf.reduce_any(tf.not_equal(best_centroids,cluster_assignments))

In [73]:
def bucket_mean(data, bucket_ids, num_buckets):
    total = tf.unsorted_segment_sum(data,bucket_ids,num_buckets)
    count = tf.unsorted_segment_sum(tf.ones_like(data),bucket_ids,num_buckets)
    
    return total / count

In [74]:
#
means = bucket_mean(datasets,best_centroids,K)

In [75]:
with tf.control_dependencies([did_assignment_changed]):
    do_updates = tf.group(centroids.assign(means),cluster_assignments.assign(best_centroids))

In [76]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [77]:
changed = True
iter = 0

In [78]:
while changed and iter < Max_iter:
    iter+=1
    [changed,_] = sess.run([did_assignment_changed,do_updates])
    print(iter)
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [79]:
[centers,assignment] = sess.run([centroids,cluster_assignments])

In [80]:
print("Centroids:")
print (centers)
print ("Cluster assignments:", assignment)

Centroids:
[[  5.55655496   5.02433244   0.7866879    0.33310551   2.25607542
   20.3879374   18.69900168   9.93225804  10.67592951  14.46502012
    4.42633535   2.16189381   3.29802055   1.73256891   3.49976094
    4.22711569   0.60397618   2.26334767   0.2971026    0.63915918
    4.35975621   3.76676188   3.13131426   5.12751532   5.88056227
    5.1182315    0.25790372   4.75814599   3.87224984   2.95905493
    1.53952337   4.05926761   2.47987815   2.28091154   4.28850966
    1.52763658   1.53280056   3.15267835   1.15815066   2.73773025
    5.02182094   4.00911891  24.57940229  21.3960366   16.12759025
    3.69055202   3.56703356   4.24332916   2.37159767   3.70762511]
 [  5.605914     4.831002     0.826025     0.38635      2.098365    18.323625
   18.974975    10.1903       9.1745      13.93938      4.42556      2.23681
    2.98403      1.943312     4.62487      4.25147      0.537042     2.232192
    0.27457      0.743        4.410275     3.94684      2.71146      5.379468
    6.2

In [81]:
#tf.contrib.learn.KMeansClustering(num_clusters=10,initial_clusters=,distance_metric=,use_mini_batch=True,mini_batch_steps_per_iteration=1,kmeans_plus_plus_num_retries=2)